# use class to encapsulate scraping functions

2021/01/03

In [ ]:
from STEP0_paths_open_save import open_files, save_files
from STEP1_scraping_cleaning_merge import scrape, app_detail_dicts
from STEP2_pre_processing import pre_processing
from STEP3_NLP_pipeline import nlp_pipeline

# 1. Scraping

In [ ]:
initial_panels = ['201812', '201907', '201908', '201912', '202001', '202002']
current_panel = '202102'

for i in initial_panels:
    o = open_files(initial_panel=i)
    DF = o.open_df_to_id_for_scraping()
    new_class_obj = scrape(data_before_202009=DF)
    print('start scraping apps with initial panel', i)
    app_details = new_class_obj.scraping_apps_according_to_id()
    print('Finished scraping app details with initial panel', i)
    s = save_files(initial_panel=i,
                   current_panel=current_panel,
                   app_details_dict=app_details)
    s.save_scraped_app_details()
    print('Saved scarped app details with initial panel', i)

# 3. Convert

In [3]:
%run -i 'STEP0_paths_open_save.py'
%run -i 'STEP1_scraping_cleaning_merge.py'
# key represents initial panel, value represents panels (including the initial) tracking the initial panel
panels_track = {
'201812':['201812', '201912', '202001', '202003', '202004', '202009', '202010',
          '202011', '202012', '202101', '202102'],
'201907':['201907', '201908', '201909', '201912', '202001', '202003', '202004',
          '202009', '202010', '202011', '202012', '202101', '202102'],
'201908':['201908', '201912', '202001', '202003', '202004', '202009', '202010',
          '202011', '202012', '202101', '202102'],
'201912':['201912', '202001', '202003', '202004', '202009', '202010', '202011',
          '202012', '202101', '202102'],
'202001':['202001', '202003', '202004', '202009', '202010', '202011', '202012',
          '202101', '202102'],
'202002':['202002', '202004', '202009', '202010', '202011', '202012', '202101',
          '202102']}

for initial_panel, all_panels in panels_track.items():
    o = open_files(initial_panel=initial_panel,
                   all_panels=all_panels)
    app_details = o.open_initial_panel_with_its_tracking_panels()
    A = app_detail_dicts(d=app_details, all_panels=all_panels)
    merged_df = A.merge_panels_into_single_df()
    save = save_files(initial_panel=initial_panel, df=merged_df)
    save.save_panel_df(name='MERGED')
    print('panel data', initial_panel, 'has shape:')
    print(merged_df.shape)

panel data 201812 has shape:
(4278, 292)
panel data 201907 has shape:
(10576, 344)
panel data 201908 has shape:
(10537, 292)
panel data 201912 has shape:
(7058, 266)
panel data 202001 has shape:
(7027, 240)
panel data 202002 has shape:
(6860, 214)


# 4. Convert to Developer Index and Geocode Addresses

In [ ]:
from tqdm.auto import tqdm
tqdm.pandas()
%run -i 'STEP0_paths_open_save.py'
%run -i 'STEP1_scraping_cleaning_merge.py'
%run -i 'STEP2_pre_processing.py'

panels_track = {
'201812':['201812', '201912', '202001', '202003', '202004', '202009', '202010',
          '202011', '202012', '202101', '202102'],
'201907':['201907', '201908', '201909', '201912', '202001', '202003', '202004',
          '202009', '202010', '202011', '202012', '202101', '202102'],
'201908':['201908', '201912', '202001', '202003', '202004', '202009', '202010',
          '202011', '202012', '202101', '202102'],
'201912':['201912', '202001', '202003', '202004', '202009', '202010', '202011',
          '202012', '202101', '202102'],
'202001':['202001', '202003', '202004', '202009', '202010', '202011', '202012',
          '202101', '202102'],
'202002':['202002', '202004', '202009', '202010', '202011', '202012', '202101',
          '202102']}
consec_panels = ['202009', '202010', '202011', '202012', '202101', '202102']
# ***************************************************************************************
for initial_panel, all_panels in panels_track.items():
    o = open_files(initial_panel=initial_panel)
    merged_df = o.open_panel_df(name='MERGED')
    new_class_obj = pre_processing(
                     df=merged_df,
                     df_index = 'appid',
                     initial_panel=initial_panel,
                     all_panels=all_panels,
                     consec_panels=consec_panels)
    DF = new_class_obj.lat_and_long_columns(
                    multiindex=False,
                    consecutive=False)
    s = save_files(df=DF, initial_panel=initial_panel)
    s.save_panel_df(name='dev_index_geocoded')
    print('finished converting', initial_panel, 'dataset')
    print()
    print()

# 5. Impute Missing

In [2]:
%run -i 'STEP0_paths_open_save.py'
%run -i 'STEP1_scraping_cleaning_merge.py'
%run -i 'STEP2_pre_processing.py'

all_init_panels = ['201812', '201907', '201908', '201912', '202001', '202002']
panels_track = {
'201812':['201812', '201912', '202001', '202003', '202004', '202009', '202010',
          '202011', '202012', '202101', '202102'],
'201907':['201907', '201908', '201909', '201912', '202001', '202003', '202004',
          '202009', '202010', '202011', '202012', '202101', '202102'],
'201908':['201908', '201912', '202001', '202003', '202004', '202009', '202010',
          '202011', '202012', '202101', '202102'],
'201912':['201912', '202001', '202003', '202004', '202009', '202010', '202011',
          '202012', '202101', '202102'],
'202001':['202001', '202003', '202004', '202009', '202010', '202011', '202012',
          '202101', '202102'],
'202002':['202002', '202004', '202009', '202010', '202011', '202012', '202101',
          '202102']}
consec_panels = ['202009', '202010', '202011', '202012', '202101', '202102']

vars_to_count_missing = ['minInstalls', 'score', 'reviews',
                          'ratings', 'price', 'description',
                          'released', 'updated', 'size', 'free',
                          'containsAds', 'offersIAP', 'contentRating',
                          'genreId']
imputation_block = {'minInstalls':[2, 'mean'],
                       'score':[4, 'mean'],
                       'reviews':[4, 'mean'],
                       'ratings':[4, 'mean'],
                       'price':[4, 'mean'],
                       'size':[7, 'mode'],
                       'released':[4, 'mode'],
                       'updated':[1, 'previous'],
                       'free':[1, 'previous'],
                       'containsAds':[7, 'mode'],
                       'offersIAP':[7, 'mode'],
                       'contentRating':[7, 'mode'],
                       'genreId':[7,'mode']}
# updated value has to strictly follow the previous panel's
# ***************************************************************************************
for initial_panel, all_panels in panels_track.items():
    o = open_files(initial_panel=initial_panel)
    merged_df = o.open_panel_df(name='MERGED')
    new_class_obj = pre_processing(
                 df=merged_df,
                 df_index = 'appid',
                 initial_panel=initial_panel,
                 all_panels=all_panels,
                 consec_panels=consec_panels)
    # --------------- check missing before imputing --------------------------------------
    missingdf = new_class_obj.count_missing(var_list=vars_to_count_missing)
    s = save_files(df=missingdf, initial_panel=initial_panel)
    s.save_df_to_csv(name='missing_count_before_imputation')
    print(missingdf)
    # --------------- IMPUTING ----------------------------------------------------------
    cols_to_impute = []
    for c, specs in imputation_block.items():
        imputed_cols = new_class_obj.impute_missing_using_adj_panels(
            var=c,
            adj_panels=specs[0],
            method=specs[1])
        cols_to_impute.append(imputed_cols)
    IMPUTED_DF = new_class_obj.replace_cols_list(list_new_cols=cols_to_impute)
    # ----------------- check missing after imputing ----------------------------------
    new_class_obj = pre_processing(
             df=IMPUTED_DF,
             df_index = 'appid',
             initial_panel=initial_panel,
             all_panels=all_panels,
             consec_panels=consec_panels)
    missingdf = new_class_obj.count_missing(var_list=vars_to_count_missing)
    s = save_files(df=missingdf, initial_panel=initial_panel)
    s.save_df_to_csv(name='missing_count_after_imputation')
    print(missingdf)
    # ------------------ save ---------------------------------------------------------
    s = save_files(df=IMPUTED_DF, initial_panel=initial_panel)
    s.save_panel_df(name='imputed_missing')
    print('finished imputing missing for', initial_panel, 'dataset')
    print()
    print()

                      count missing
minInstalls_201812    15           
minInstalls_201912    15           
minInstalls_202001    15           
minInstalls_202003    15           
minInstalls_202004    15           
minInstalls_202009    16           
minInstalls_202010    62           
minInstalls_202011    112          
minInstalls_202012    190          
minInstalls_202101    225          
minInstalls_202102    266          
score_201812          12           
score_201912          12           
score_202001          12           
score_202003          12           
score_202004          12           
score_202009          26           
score_202010          72           
score_202011          122          
score_202012          200          
score_202101          235          
score_202102          276          
reviews_201812        12           
reviews_201912        12           
reviews_202001        12           
reviews_202003        12           
reviews_202004        12    

# 6. Delete Missing

In [3]:
%run -i 'STEP0_paths_open_save.py'
%run -i 'STEP1_scraping_cleaning_merge.py'
%run -i 'STEP2_pre_processing.py'

all_init_panels = ['201812', '201907', '201908', '201912', '202001', '202002']
panels_track = {
'201812':['201812', '201912', '202001', '202003', '202004', '202009', '202010',
          '202011', '202012', '202101', '202102'],
'201907':['201907', '201908', '201909', '201912', '202001', '202003', '202004',
          '202009', '202010', '202011', '202012', '202101', '202102'],
'201908':['201908', '201912', '202001', '202003', '202004', '202009', '202010',
          '202011', '202012', '202101', '202102'],
'201912':['201912', '202001', '202003', '202004', '202009', '202010', '202011',
          '202012', '202101', '202102'],
'202001':['202001', '202003', '202004', '202009', '202010', '202011', '202012',
          '202101', '202102'],
'202002':['202002', '202004', '202009', '202010', '202011', '202012', '202101',
          '202102']}
consec_panels = ['202009', '202010', '202011', '202012', '202101', '202102']
vars_to_count_missing = ['minInstalls', 'score', 'reviews',
                          'ratings', 'price', 'description',
                          'released', 'size', 'free',
                          'containsAds', 'offersIAP', 'contentRating', 'genreId']
deletion_block = {'minInstalls':2,
                       'score':2,
                       'reviews':2,
                       'ratings':2,
                       'price':2,
                       'size':2,
                       'released':2,
                       'updated':2,
                       'free':1,
                       'containsAds':2,
                       'offersIAP':2,
                       'contentRating':2,
                       'genreId':2}
# ***************************************************************************************
for initial_panel, all_panels in panels_track.items():
    o = open_files(initial_panel=initial_panel)
    merged_df = o.open_panel_df(name='imputed_missing')
    new_class_obj = pre_processing(
                 df=merged_df,
                 df_index = 'appid',
                 initial_panel=initial_panel,
                 all_panels=all_panels,
                 consec_panels=consec_panels)
    appids_to_remove = []
    for var, n in deletion_block.items():
        missing_df, missing_appids = new_class_obj.check_apps_with_consecutive_missing_panels(
        var=var,
        number_consec_panels_missing=n)
        appids_to_remove.extend(missing_appids)
    print('total', len(appids_to_remove), 'to remove')
    appids_to_remove = list(set(appids_to_remove))
    print('total unique', len(appids_to_remove), 'to remove')
    print('before remove rows', initial_panel, 'has shape', new_class_obj.df.shape)
    new_df = new_class_obj.drop_rows(list_of_row_labels=appids_to_remove)
    print('after remove rows', initial_panel, 'has shape', new_df.shape)
    # ----------------- check missing after deletion ----------------------------------
    new_class_obj = pre_processing(
         df=new_df,
         df_index = 'appid',
         initial_panel=initial_panel,
         all_panels=all_panels,
         consec_panels=consec_panels)
    missingdf = new_class_obj.count_missing(var_list=vars_to_count_missing)
    s = save_files(df=missingdf, initial_panel=initial_panel)
    s.save_df_to_csv(name='missing_count_after_imputation_and_deletion')
    print(missingdf)
    # ------------------------------ save ---------------------------------------------
    s = save_files(df=new_df, initial_panel=initial_panel)
    s.save_panel_df(name='imputed_and_deleted_missing')
    print('finished imputing and deleting missing for', initial_panel, 'dataset')
    print()
    print()

number of apps with at least 2 consecutive missing panels for minInstalls are 181
out of 4278 apps.
number of apps with at least 2 consecutive missing panels for score are 62
out of 4278 apps.
number of apps with at least 2 consecutive missing panels for reviews are 62
out of 4278 apps.
number of apps with at least 2 consecutive missing panels for ratings are 429
out of 4278 apps.
number of apps with at least 2 consecutive missing panels for price are 52
out of 4278 apps.
number of apps with at least 2 consecutive missing panels for size are 429
out of 4278 apps.
number of apps with at least 2 consecutive missing panels for released are 120
out of 4278 apps.
number of apps with at least 2 consecutive missing panels for updated are 172
out of 4278 apps.
number of apps with at least 2 consecutive missing panels for free are 172
out of 4278 apps.
number of apps with at least 2 consecutive missing panels for containsAds are 2
out of 4278 apps.
number of apps with at least 2 consecutive mis

# 6. NLP Pipeline

In [3]:
%run -i 'STEP0_paths_open_save.py'
%run -i 'STEP3_NLP_pipeline.py'

all_init_panels = ['201812', '201907', '201908', '201912', '202001', '202002']
panels_track = {
'201812':['201812', '201912', '202001', '202003', '202004', '202009', '202010',
          '202011', '202012', '202101', '202102'],
'201907':['201907', '201908', '201909', '201912', '202001', '202003', '202004',
          '202009', '202010', '202011', '202012', '202101', '202102'],
'201908':['201908', '201912', '202001', '202003', '202004', '202009', '202010',
          '202011', '202012', '202101', '202102'],
'201912':['201912', '202001', '202003', '202004', '202009', '202010', '202011',
          '202012', '202101', '202102'],
'202001':['202001', '202003', '202004', '202009', '202010', '202011', '202012',
          '202101', '202102'],
'202002':['202002', '202004', '202009', '202010', '202011', '202012', '202101',
          '202102']}
consec_panels = ['202009', '202010', '202011', '202012', '202101', '202102']

# ***************************************************************************************
for initial_panel, all_panels in panels_track.items():
    o = open_files(initial_panel=initial_panel)
    merged_df = o.open_panel_df(name='MERGED')
    new_class_obj = nlp_pipeline(
                      df=merged_df,
                      text_col_name='description',
                      initial_panel=initial_panel,
                      all_panels=all_panels,
                      consec_panels=consec_panels)
    new_df = new_class_obj.add_predicted_cluster_labels_to_df(
        n_clusters=200,
        init='k-means++',
        random_state=50,
        n_comp=1500,
        consecutive=False,
        combine_panels=False)
    # ------------------------------ save ---------------------------------------------
    s = save_files(df=new_df, initial_panel=initial_panel)
    s.save_panel_df(name='predicted_text_cluster_labels')
    print('finished predicting text cluster labels for', initial_panel, 'dataset')
    print()
    print()

<class 'scipy.sparse.csr.csr_matrix'>
(4278, 1500)
finished predicting text cluster labels for 201812 dataset


<class 'scipy.sparse.csr.csr_matrix'>
(10576, 1500)
finished predicting text cluster labels for 201907 dataset


<class 'scipy.sparse.csr.csr_matrix'>
(10537, 1500)
finished predicting text cluster labels for 201908 dataset


<class 'scipy.sparse.csr.csr_matrix'>
(7058, 1500)
finished predicting text cluster labels for 201912 dataset


<class 'scipy.sparse.csr.csr_matrix'>
(7027, 1500)
finished predicting text cluster labels for 202001 dataset


<class 'scipy.sparse.csr.csr_matrix'>
(6860, 1500)
finished predicting text cluster labels for 202002 dataset


